In [1]:
"""
@Josh Boquiren
OTIS Lab MUSC
4.5.2023

README: 
-simple program to plot all polylines on the same graph

"""

'\n@Josh Boquiren\nOTIS Lab MUSC\n4.5.2023\n\nREADME: \n-simple program to plot all polylines on the same graph\n\n'

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#initialize directories

basedir = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data'

earlyACQ = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data/EarlyAcq'
middleACQ = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data/MidAcq'
lateACQ = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data\LateAcq'

earlyEXT = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data/EarlyExt'
lateEXT = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data\LastExt'

CUE = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data/CueRein'
DRUG = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data\DrugRein'
TMT = r'C:\Users\jboqu\Dropbox\2P Imaging Projects\Beth\Josh\Project Datasets\PFC Self-Admin Analysis\PFC Self Admin Data\TMTRein'


In [11]:
#ACQ aoi's
early_acq_aoi = [
    'CTL1',
    'ER-L1','ER-L2',
    'IG-19',
    'LCDD-PGa1','LCDD-PGa3','LCDD-PGa4','LCDD-PGa5','LCDD-PGa6',
    'LCDD-PGa-T1','LCDD-PGa-T2','LCDD-PGa-T3','LCDD-PGa-T4','LCDD-PGa-T5',
    'PGa-T1','PGa-T2','PGa-T3'
    ]  
middle_acq_aoi = [
    'CTL1',
    'ER-L1','ER-L2',
    'IG-19',
    'LCDD-PGa1','LCDD-PGa3','LCDD-PGa4','LCDD-PGa5','LCDD-PGa6',
    'LCDD-PGa-T1','LCDD-PGa-T2','LCDD-PGa-T3','LCDD-PGa-T4','LCDD-PGa-T5',
    'PGa-T1','PGa-T2','PGa-T3'
    ]  
late_acq_aoi = [
    'CTL1',
    'ER-L1','ER-L2',
    'IG-19',
    'LCDD-PGa1','LCDD-PGa3','LCDD-PGa4','LCDD-PGa5','LCDD-PGa6',
    'LCDD-PGa-T1','LCDD-PGa-T2','LCDD-PGa-T3','LCDD-PGa-T4','LCDD-PGa-T5',
    'PGa-T1','PGa-T2','PGa-T3'
    ]  

#EXT aoi's
early_ext_aoi = [
    'CTL1',
    'ER-L1','ER-L2',
    'LCDD-PGa1','LCDD-PGa4','LCDD-PGa6',
    'LCDD-PGa-T1','LCDD-PGa-T3','LCDD-PGa-T4',
    'PGa-T1','PGa-T2','PGa-T3'
    ]  
middle_ext_aoi = [ #really late animals, but coded as middle
    'CTL1',
    'ER-L1','ER-L2',
    'LCDD-PGa1','LCDD-PGa5',
    'LCDD-PGa-T2','LCDD-PGa-T3','LCDD-PGa-T4','LCDD-PGa-T5',
    'PGa-T1','PGa-T2'
    ]  
late_ext_aoi = []


#RST aoi's
early_rst_aoi = [
    'LCDD-PGa4','LCDD-PGa5','LCDD-PGa6',
    'PGa-T1','PGa-T2','PGa-T3'
    ]  
middle_rst_aoi = [
    'LCDD-PGa4','LCDD-PGa6',
    'PGa-T1','PGa-T2' #omit 'PGa-T3','LCDD-PGa5' from CNO
    ]  
late_rst_aoi = [
    'LCDD-PGa4','LCDD-PGa6',
    'PGa-T1','PGa-T2','PGa-T3'#,'LCDD-PGa5'
    ] 

In [5]:
#frame rate variables
frameaveraging = 4
timebetweenframes = 33.3333
framerate = 30
framerate = timebetweenframes/frameaveraging #raw frame rate
averagedframerate = timebetweenframes/frameaveraging #averaged frame rate

#window size variables
pre_window_size = int(10*framerate) #How many frames per trial before origin to be plotted?
window_size =  int((pre_window_size*2)+(3*framerate)) #How many frames do you want to plot around the origin?
post_window_size = window_size - pre_window_size
baselinefirstframe = 0
baselinelastframe = int(1*framerate)
infusionframe = int(pre_window_size+(3*framerate))

In [12]:
"""ACQUISITION"""
#initializes np array to stack on top of
population_data_ACQ = np.nan*np.ones((1, window_size))
for animal in early_acq_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(earlyACQ, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(earlyACQ, animal, fov, "active_data.npy"))
                population_data_ACQ = np.vstack((population_data_ACQ, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_early_ACQ = population_data_ACQ.shape[0]-1 

#slices population data to not include first row
population_data_ACQ = population_data_ACQ[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_ACQ[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_ACQ = population_data_ACQ - baseline[:,None]

#other variables
numrois_early_active = population_data_ACQ.shape[0]
tempresponse = np.nanmean(population_data_ACQ[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_early_ACQ = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
early_ACQ = population_data_ACQ

#initializes np array to stack on top of
population_data_ACQ = np.nan*np.ones((1, window_size))
for animal in middle_acq_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(middleACQ, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(middleACQ, animal, fov, "active_data.npy"))
                population_data_ACQ = np.vstack((population_data_ACQ, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_middle_ACQ = population_data_ACQ.shape[0]-1 

#slices population data to not include first row
population_data_ACQ = population_data_ACQ[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_ACQ[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_ACQ = population_data_ACQ - baseline[:,None]

#other variables
numrois_middle_active = population_data_ACQ.shape[0]
tempresponse = np.nanmean(population_data_ACQ[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_middle_ACQ = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
middle_ACQ = population_data_ACQ

#initializes np array to stack on top of
population_data_ACQ = np.nan*np.ones((1, window_size))
for animal in late_acq_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(lateACQ, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(lateACQ, animal, fov, "active_data.npy"))
                population_data_ACQ = np.vstack((population_data_ACQ, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_late_ACQ = population_data_ACQ.shape[0]-1 

#slices population data to not include first row
population_data_ACQ = population_data_ACQ[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_ACQ[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_ACQ = population_data_ACQ - baseline[:,None]

#other variables
numrois_late_active = population_data_ACQ.shape[0]
tempresponse = np.nanmean(population_data_ACQ[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_late_active = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
late_ACQ = population_data_ACQ



"""EXTINCTION"""
#initializes np array to stack on top of
population_data_EXT = np.nan*np.ones((1, window_size))
for animal in early_ext_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(earlyEXT, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(earlyEXT, animal, fov, "active_data.npy"))
                population_data_EXT = np.vstack((population_data_EXT, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_early_EXT = population_data_EXT.shape[0]-1 

#slices population data to not include first row
population_data_EXT = population_data_EXT[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_EXT[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_EXT = population_data_EXT - baseline[:,None]

#other variables
numrois_early_active = population_data_EXT.shape[0]
tempresponse = np.nanmean(population_data_EXT[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_early_EXT = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
early_EXT = population_data_EXT

#initializes np array to stack on top of
population_data_EXT = np.nan*np.ones((1, window_size))
for animal in middle_ext_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(lateEXT, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(lateEXT, animal, fov, "active_data.npy"))
                population_data_EXT = np.vstack((population_data_EXT, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_middle_EXT = population_data_ACQ.shape[0]-1 

#slices population data to not include first row
population_data_EXT = population_data_EXT[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_EXT[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_ACQ = population_data_EXT - baseline[:,None]

#other variables
numrois_middle_active = population_data_EXT.shape[0]
tempresponse = np.nanmean(population_data_ACQ[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_middle_EXT = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
middle_EXT = population_data_EXT



"""REINSTATEMENT"""
#initializes np array to stack on top of
population_data_CUE = np.nan*np.ones((1, window_size))
for animal in early_rst_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(CUE, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(CUE, animal, fov, "active_data.npy"))
                population_data_CUE = np.vstack((population_data_CUE, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_early_CUE = population_data_CUE.shape[0]-1 

#slices population data to not include first row
population_data_CUE = population_data_CUE[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_CUE[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_CUE = population_data_CUE - baseline[:,None]

#other variables
numrois_early_active = population_data_CUE.shape[0]
tempresponse = np.nanmean(population_data_CUE[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_early_CUE = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
early_CUE = population_data_CUE

#initializes np array to stack on top of
population_data_DRUG = np.nan*np.ones((1, window_size))
for animal in middle_rst_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(DRUG, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(DRUG, animal, fov, "active_data.npy"))
                population_data_DRUG = np.vstack((population_data_DRUG, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_middle_DRUG = population_data_DRUG.shape[0]-1 

#slices population data to not include first row
population_data_DRUG = population_data_DRUG[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_DRUG[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_DRUG = population_data_DRUG - baseline[:,None]

#other variables
numrois_middle_active = population_data_DRUG.shape[0]
tempresponse = np.nanmean(population_data_DRUG[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_middle_DRUG = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
middle_DRUG = population_data_DRUG

#initializes np array to stack on top of
population_data_TMT = np.nan*np.ones((1, window_size))
for animal in late_rst_aoi:
    print('>>>', animal)
    FOVs = next(os.walk(os.path.join(TMT, animal)))[1]
    for fov in sorted(FOVs):
        #load in data
        try:
                active_data = np.load(os.path.join(TMT, animal, fov, "active_data.npy"))
                population_data_ACQ = np.vstack((population_data_TMT, active_data))
        except:
               print('***ERROR: no files!***')

#num neurons is length of data - 1
numneurons_late_TMT = population_data_TMT.shape[0]-1 

#slices population data to not include first row
population_data_TMT = population_data_TMT[1:,:]

#calculating population active data - baseline
baseline = np.nanmean(population_data_TMT[:,baselinefirstframe:baselinelastframe], axis=1)
population_data_TMT = population_data_TMT - baseline[:,None]

#other variables
numrois_late_active = population_data_TMT.shape[0]
tempresponse = np.nanmean(population_data_TMT[:,pre_window_size-(1*int(framerate)):pre_window_size+1*int(framerate)], axis=1)
sortresponse_late_active = np.argsort(tempresponse)[::-1]

#creating new variables for combined early active and inactive data because pop_active/pop_inactive will be overwritten
late_TMT = population_data_TMT

>>> CTL1
>>> ER-L1
>>> ER-L2
>>> IG-19
>>> LCDD-PGa1
>>> LCDD-PGa3
>>> LCDD-PGa4
>>> LCDD-PGa5
>>> LCDD-PGa6
>>> LCDD-PGa-T1
>>> LCDD-PGa-T2
>>> LCDD-PGa-T3
>>> LCDD-PGa-T4
>>> LCDD-PGa-T5
>>> PGa-T1
>>> PGa-T2
>>> PGa-T3
>>> CTL1
>>> ER-L1
>>> ER-L2
>>> IG-19
>>> LCDD-PGa1
>>> LCDD-PGa3
>>> LCDD-PGa4
>>> LCDD-PGa5
>>> LCDD-PGa6
>>> LCDD-PGa-T1
>>> LCDD-PGa-T2
>>> LCDD-PGa-T3
>>> LCDD-PGa-T4
>>> LCDD-PGa-T5
>>> PGa-T1
>>> PGa-T2
>>> PGa-T3
>>> CTL1
>>> ER-L1
>>> ER-L2
>>> IG-19
>>> LCDD-PGa1
>>> LCDD-PGa3
>>> LCDD-PGa4
>>> LCDD-PGa5
>>> LCDD-PGa6
>>> LCDD-PGa-T1
>>> LCDD-PGa-T2
>>> LCDD-PGa-T3
>>> LCDD-PGa-T4
>>> LCDD-PGa-T5
>>> PGa-T1
>>> PGa-T2
>>> PGa-T3
>>> CTL1
>>> ER-L1
>>> ER-L2
>>> LCDD-PGa1
>>> LCDD-PGa4
>>> LCDD-PGa6
>>> LCDD-PGa-T1
>>> LCDD-PGa-T3
>>> LCDD-PGa-T4
>>> PGa-T1
>>> PGa-T2
>>> PGa-T3
>>> CTL1
>>> ER-L1
>>> ER-L2
>>> LCDD-PGa1
>>> LCDD-PGa5
>>> LCDD-PGa-T2
>>> LCDD-PGa-T3
>>> LCDD-PGa-T4
>>> LCDD-PGa-T5
>>> PGa-T1
>>> PGa-T2
>>> LCDD-PGa4
>>> LCDD-PGa5
>>> LCDD-PG

In [ ]:
early_populationdata = early_active
middle_populationdata = middle_active
late_populationdata = late_active
all_populationdata = np.vstack((early_active, middle_active, late_active))

all_newlabels = np.hstack((early_newlabels, middle_newlabels, late_newlabels))

early_numneurons = early_populationdata.shape[0]
middle_numneurons = middle_populationdata.shape[0]
late_numneurons = late_populationdata.shape[0]
all_numneurons = all_populationdata.shape[0]

sessions = ['early', 'middle', 'late', 'all'] ###SET TO THE DIFFERENT TIME POINTS THAT WERE USED FOR TRACKING
numclusters = 4 ###SET TO NUMBER OF CLUSTERS FOR DATASET
uniquelabels = np.arange(numclusters)

sortwindow = {}
for c, cluster in enumerate(uniquelabels):
    sortwindow[c] = {}
    if c == 0:
        sortwindow[c] = [pre_window_size, infusionframe+int(3*framerate)]
    if c == 1:
        sortwindow[c] = [infusionframe+int(1*framerate), -1]
    if c == 2:
        sortwindow[c] = [pre_window_size, infusionframe+int(3*framerate)]
    if c == 3:
        sortwindow[c] = [pre_window_size, infusionframe+int(3*framerate)]
    
cmax = 0.1

fig, axs = plt.subplots(len(sessions),len(uniquelabels),
                        figsize=(2*len(uniquelabels),2*len(sessions)))
cbar_ax = fig.add_axes([.94, .3, .01, .4])
cbar_ax.tick_params(width=0.5) 

numroisincluster = np.nan*np.ones((len(uniquelabels),))

for c, cluster in enumerate(uniquelabels):
    for s, session in enumerate(sessions):
        if session == 'early':
            temp = early_populationdata[np.where(early_newlabels==cluster)[0],:]
        elif session == 'middle':
            temp = middle_populationdata[np.where(middle_newlabels==cluster)[0],:]
        elif session == 'late':
            temp = late_populationdata[np.where(late_newlabels==cluster)[0],:]
        elif session == 'all':
            temp = all_populationdata[np.where(all_newlabels==cluster)[0],:]
        numroisincluster[c] = temp.shape[0]
        sortresponse = np.argsort(np.mean(temp[:,sortwindow[c][0]:sortwindow[c][1]], axis=1))[::-1]
        hm = sns.heatmap(
                    temp[sortresponse],
                    ax=axs[s, cluster],
                    cmap=sns.diverging_palette(154, 308,n=230,s=100,sep=50,center='dark'),
                    vmin=-cmax,
                    vmax=cmax,
                    cbar=(cluster==0),
                    cbar_ax=cbar_ax if (cluster==0) else None,
                    cbar_kws={'label': 'Normalized fluorescence', 'ticks': [cmax, cmin]}
                    )

        axs[s, cluster].grid(False)
        if s==len(sessions)-1:
            axs[s, cluster].set_xticks([0, pre_window_size,
                                        infusionframe, window_size])
        else:
            axs[s, cluster].set_xticks([])
        axs[s, cluster].tick_params(width=0.5)    
        axs[s, cluster].set_xticklabels([])
        axs[s, cluster].set_yticks([])
        axs[s, cluster].axvline(pre_window_size, linestyle='--', color='k', linewidth=1.5)
        axs[s, cluster].axvline(infusionframe, linestyle='--', color='k', linewidth=1.5)
        if cluster==0:
            axs[s, 0].set_ylabel('%s\nNeurons'%(session))
    axs[0, cluster].set_title('Cluster %d'%(cluster+1))

fig.text(0.5, 0.05, 'Time (sec)', fontsize=12,
         horizontalalignment='center', verticalalignment='center', rotation='horizontal')
fig.tight_layout()

fig.subplots_adjust(wspace=0.1, hspace=0.1)
fig.subplots_adjust(left=0.03)
fig.subplots_adjust(right=0.93)
fig.subplots_adjust(bottom=0.2)
fig.subplots_adjust(top=0.83)

# fig.savefig(os.path.join(basedir, 'Results', 'PFC HEROIN SA - CueDrugTMT, NoCueOnly Ensemble Heatmaps.PDF'), format='PDF')
# fig.savefig(os.path.join(basedir, 'Results', 'PFC HEROIN SA - CueDrugTMT, NoCueOnly Ensemble Heatmaps.png'), format='PNG')



In [ ]:
fig, axs = plt.subplots(len(sessions), len(uniquelabels),
                        figsize=(2*len(uniquelabels),3*len(sessions)))

numroisincluster = np.nan*np.ones((len(uniquelabels),))

colors_for_key = ['red','blue','purple','green']

for c, cluster in enumerate(uniquelabels):
    for s, session in enumerate(sessions):
        if session == 'early':
            temp = early_populationdata[np.where(early_newlabels==cluster)[0],:]
        elif session == 'middle':
            temp = middle_populationdata[np.where(middle_newlabels==cluster)[0],:]
        elif session == 'late':
            temp = late_populationdata[np.where(late_newlabels==cluster)[0],:]
        elif session == 'all':
            temp = all_populationdata[np.where(all_newlabels==cluster)[0],:]
        numroisincluster[c] = temp.shape[0]
        temp = np.mean(temp, axis = 0)        
        sns.lineplot(data = temp, dashes = False, color = colors_for_key[s],
                    ax=axs[s, c])

        axs[s, cluster].grid(False)
        if s==len(sessions)-1:
            axs[s, cluster].set_xticks([0, pre_window_size, infusionframe, window_size])
        else:
            axs[s, cluster].set_xticks([])
        axs[s, cluster].tick_params(width=0.5)    
        axs[s, cluster].set_yticks([])
        axs[s, cluster].set(ylim=(-.2, .2))
        axs[s, cluster].text(x = s, y = .1, s = 'n = %d'%(numroisincluster[c]))
        
        axs[s, cluster].axvline(pre_window_size, linestyle='--', color='k', linewidth=1.5)
        axs[s, cluster].axvline(infusionframe, linestyle='--', color='k', linewidth=1.5)
        axs[s, cluster].axhline(0, linestyle='--', color='k', linewidth=0.5)
        if cluster==0:
            axs[s, 0].set_ylabel(session)
    axs[0, cluster].set_title('Cluster %d'%(cluster+1))
    
fig.text(0.5, 0.05, 'Time from cue (s)', fontsize=12,horizontalalignment='center', verticalalignment='center', rotation='horizontal')
fig.tight_layout()

fig.subplots_adjust(wspace=0.1, hspace=0.1)
fig.subplots_adjust(left=0.03)
fig.subplots_adjust(right=0.93)
fig.subplots_adjust(bottom=0.2)
fig.subplots_adjust(top=0.83)

# fig.savefig(os.path.join(basedir, 'Results', 'PFC HEROIN SA - CueDrugTMT, NoCueOnly Ensemble Lines.PDF'), format='PDF')
# fig.savefig(os.path.join(basedir, 'Results', 'PFC HEROIN SA - CueDrugTMT, NoCueOnly Ensemble Lines.png'), format='PNG')
